In [1]:
#dependencies

import pandas as pd
import requests
import json
from pprint import pprint
from config import lastfm_api

In [2]:
#Create Top Love Songs DataFrame to populate with info retrieved from LastFM

top_songs_df=pd.DataFrame({
    "Rank": [],
    "Song Name": [],
    "Artist": [],
    "Duration": [],
    "Album": []
})
top_songs_df

,Rank,Song Name,Artist,Duration,Album


In [3]:
#INSTRUCTIONS FROM WEBSITE
    #note: Use UTF-8 to encode arguments when calling API methods.
    #limit (Optional) : The number of results to fetch per page. Defaults to 50.
    #page (Optional) : The page number to fetch. Defaults to first page.

#Set the parameters to LastFM API calls. Method and Limit are empty so we can adjust them to specific calls\
#like get_top_tracks or track.getInfo

params={
    "tag": "love",
    "api_key": lastfm_api,
    "format": "json",
    "method":"",
    "limit": ""
}

base_url= "http://ws.audioscrobbler.com/2.0/?"
example_url= "/2.0/?method=tag.gettoptracks&tag=disco&api_key=YOUR_API_KEY&format=json"    #taken from website


In [4]:
#API call for 1000 songs using tag.getTopTracks
params["limit"]=1000
params["method"]= "tag.gettoptracks"

response=requests.get(base_url, params=params).json()
# pprint(response)

#The 1000 songs are in a list under "response["tracks"]["track"]". I generated a df so I can iterate through the rows to\
#retrieve info
#This way it's also easier to retrieve urls for secondary API calls

results_df=pd.DataFrame(response["tracks"]["track"])
results_df.head()

,name,duration,mbid,url,streamable,artist,image,@attr
0,Wonderwall,259,31623cce-9717-4513-9d83-1b5d04e44f9b,https://www.last.fm/music/Oasis/_/Wonderwall,"{'#text': '0', 'fulltrack': '0'}","{'name': 'Oasis', 'mbid': 'ecf9f3a3-35e9-4c58-...",[{'#text': 'https://lastfm.freetls.fastly.net/...,{'rank': '1'}
1,The Scientist,309,13f5488d-8e41-42d8-9fe9-a5295f1a9a3d,https://www.last.fm/music/Coldplay/_/The+Scien...,"{'#text': '0', 'fulltrack': '0'}","{'name': 'Coldplay', 'mbid': 'cc197bad-dc9c-44...",[{'#text': 'https://lastfm.freetls.fastly.net/...,{'rank': '2'}
2,Yellow,267,8b5bf478-22f8-4902-a1c1-0db82261db58,https://www.last.fm/music/Coldplay/_/Yellow,"{'#text': '0', 'fulltrack': '0'}","{'name': 'Coldplay', 'mbid': 'cc197bad-dc9c-44...",[{'#text': 'https://lastfm.freetls.fastly.net/...,{'rank': '3'}
3,Fix You,294,3af0af54-9c93-4578-9128-d04054286ba3,https://www.last.fm/music/Coldplay/_/Fix+You,"{'#text': '0', 'fulltrack': '0'}","{'name': 'Coldplay', 'mbid': 'cc197bad-dc9c-44...",[{'#text': 'https://lastfm.freetls.fastly.net/...,{'rank': '4'}
4,Chasing Cars,0,f62a3798-6559-4f9b-8b80-6ea3e4ad89aa,https://www.last.fm/music/Snow+Patrol/_/Chasin...,"{'#text': '0', 'fulltrack': '0'}","{'name': 'Snow Patrol', 'mbid': 'a66999a7-ae5c...",[{'#text': 'https://lastfm.freetls.fastly.net/...,{'rank': '5'}


In [7]:
#The 1000 songs are in a list under "response["tracks"]["track"]"
results=response["tracks"]["track"]

for index, row in results_df.iterrows():
    song= results[index]["name"]     
    artist= results[index]["artist"]["name"]
    print(f'Info retrieved for song:{song} by {artist}')
    
    #Retrieve info about date release of album and track duration
    try:
        song_url=f'http://ws.audioscrobbler.com/2.0/?method=track.getInfo&api_key={lastfm_api}\
                    &track={song}&artist={artist}&format=json'
        song_response=requests.get(song_url).json()
        print(f'-->Retrieving album name for {song} by {artist}')
        album=song_response["track"]["album"]["title"]

#       #Trying to retrieve album info to check release date but couldn't find any info besides wiki page
#       album_url=f'http://ws.audioscrobbler.com/2.0/?method=album.getinfo&api_key={lastfm_api}\
#                         &artist={artist}&album={album}&format=json'
#       album_response=requests.get(album_url).json()
#       pprint(album_response)
    except (KeyError, IndexError, ValueError):
        print(f'----Missing field/result for {album} by {artist}. Skipping----')
    
    #Populate df
    try:
        top_songs_df.loc[index, "Rank"]=results[index]["@attr"]["rank"]
        top_songs_df.loc[index, "Song Name"]=results[index]["name"]
        top_songs_df.loc[index, "Artist"]=results[index]["artist"]["name"]
        top_songs_df.loc[index, "Duration"]=results[index]["duration"] 
        top_songs_df.loc[index, "Album"]=song_response["track"]["album"]["title"]
            
    except (KeyError, IndexError):
        print(f'----Missing field/result for {song} by {artist}. Skipping----')


Info retrieved for song:Wonderwall by Oasis
-->Retrieving album name for Wonderwall by Oasis
Info retrieved for song:The Scientist by Coldplay
-->Retrieving album name for The Scientist by Coldplay
Info retrieved for song:Yellow by Coldplay
-->Retrieving album name for Yellow by Coldplay
Info retrieved for song:Fix You by Coldplay
-->Retrieving album name for Fix You by Coldplay
Info retrieved for song:Chasing Cars by Snow Patrol
-->Retrieving album name for Chasing Cars by Snow Patrol
Info retrieved for song:Iris by Goo Goo Dolls
-->Retrieving album name for Iris by Goo Goo Dolls
Info retrieved for song:I'm Yours by Jason Mraz
-->Retrieving album name for I'm Yours by Jason Mraz
Info retrieved for song:I Will Follow You Into The Dark by Death Cab for Cutie
-->Retrieving album name for I Will Follow You Into The Dark by Death Cab for Cutie
Info retrieved for song:How to Save a Life by The Fray
-->Retrieving album name for How to Save a Life by The Fray
Info retrieved for song:Love Will

-->Retrieving album name for Sparks by Coldplay
Info retrieved for song:Kiss Me by Sixpence None the Richer
-->Retrieving album name for Kiss Me by Sixpence None the Richer
Info retrieved for song:Let's Stay Together by Al Green
-->Retrieving album name for Let's Stay Together by Al Green
Info retrieved for song:Come Away with Me by Norah Jones
-->Retrieving album name for Come Away with Me by Norah Jones
Info retrieved for song:Lovefool by The Cardigans
-->Retrieving album name for Lovefool by The Cardigans
Info retrieved for song:This Modern Love by Bloc Party
-->Retrieving album name for This Modern Love by Bloc Party
Info retrieved for song:Cosmic Love by Florence + the Machine
-->Retrieving album name for Cosmic Love by Florence + the Machine
Info retrieved for song:Love Me Do by The Beatles
-->Retrieving album name for Love Me Do by The Beatles
Info retrieved for song:Wouldn't It Be Nice by The Beach Boys
-->Retrieving album name for Wouldn't It Be Nice by The Beach Boys
Info ret

-->Retrieving album name for Wish You Were Here by Incubus
Info retrieved for song:Blood Bank by Bon Iver
-->Retrieving album name for Blood Bank by Bon Iver
Info retrieved for song:The Blower's Daughter by Damien Rice
-->Retrieving album name for The Blower's Daughter by Damien Rice
Info retrieved for song:No One's Gonna Love You by Band of Horses
-->Retrieving album name for No One's Gonna Love You by Band of Horses
Info retrieved for song:Ocean Avenue by Yellowcard
-->Retrieving album name for Ocean Avenue by Yellowcard
Info retrieved for song:Always by blink-182
-->Retrieving album name for Always by blink-182
Info retrieved for song:Flightless Bird, American Mouth by Iron & Wine
-->Retrieving album name for Flightless Bird, American Mouth by Iron & Wine
----Missing field/result for blink-182 by Iron & Wine. Skipping----
----Missing field/result for Flightless Bird, American Mouth by Iron & Wine. Skipping----
Info retrieved for song:Bridge over Troubled Water by Simon & Garfunkel
-

-->Retrieving album name for Rehab by Rihanna
Info retrieved for song:Michelle by The Beatles
-->Retrieving album name for Michelle by The Beatles
Info retrieved for song:Suspicious Minds by Elvis Presley
-->Retrieving album name for Suspicious Minds by Elvis Presley
Info retrieved for song:Playground Love by Air
-->Retrieving album name for Playground Love by Air
Info retrieved for song:Wrecking Ball by Miley Cyrus
-->Retrieving album name for Wrecking Ball by Miley Cyrus
Info retrieved for song:Thinking of You by Katy Perry
-->Retrieving album name for Thinking of You by Katy Perry
Info retrieved for song:More Than Words by Extreme
-->Retrieving album name for More Than Words by Extreme
Info retrieved for song:Lovesong by Adele
-->Retrieving album name for Lovesong by Adele
Info retrieved for song:Holland, 1945 by Neutral Milk Hotel
-->Retrieving album name for Holland, 1945 by Neutral Milk Hotel
Info retrieved for song:When I Was Your Man by Bruno Mars
-->Retrieving album name for W

-->Retrieving album name for Time to Dance by Panic! at the Disco
Info retrieved for song:Glycerine by Bush
-->Retrieving album name for Glycerine by Bush
Info retrieved for song:Lovely Rita by The Beatles
-->Retrieving album name for Lovely Rita by The Beatles
Info retrieved for song:Ride by Lana Del Rey
-->Retrieving album name for Ride by Lana Del Rey
Info retrieved for song:I Won't Give Up by Jason Mraz
-->Retrieving album name for I Won't Give Up by Jason Mraz
Info retrieved for song:Jesus, Etc. by Wilco
-->Retrieving album name for Jesus, Etc. by Wilco
Info retrieved for song:Tiny Vessels by Death Cab for Cutie
-->Retrieving album name for Tiny Vessels by Death Cab for Cutie
Info retrieved for song:No. 1 Party Anthem by Arctic Monkeys
-->Retrieving album name for No. 1 Party Anthem by Arctic Monkeys
Info retrieved for song:Feel by Robbie Williams
-->Retrieving album name for Feel by Robbie Williams
Info retrieved for song:Please Don't Leave Me by P!nk
-->Retrieving album name for

...

-->Retrieving album name for Blue and Yellow by The Used
Info retrieved for song:Crazy Love by Van Morrison
-->Retrieving album name for Crazy Love by Van Morrison
Info retrieved for song:Nothing Lasts Forever by Maroon 5
-->Retrieving album name for Nothing Lasts Forever by Maroon 5
Info retrieved for song:Saturday by Fall Out Boy
-->Retrieving album name for Saturday by Fall Out Boy
Info retrieved for song:All alright by Sigur Rós
-->Retrieving album name for All alright by Sigur Rós
Info retrieved for song:Often by The Weeknd
-->Retrieving album name for Often by The Weeknd
Info retrieved for song:Sacrifice by Elton John
-->Retrieving album name for Sacrifice by Elton John
Info retrieved for song:She's the One by Robbie Williams
-->Retrieving album name for She's the One by Robbie Williams
Info retrieved for song:Somewhere a Clock Is Ticking by Snow Patrol
-->Retrieving album name for Somewhere a Clock Is Ticking by Snow Patrol
Info retrieved for song:New Person, Same Old Mistakes b

-->Retrieving album name for SING by My Chemical Romance
Info retrieved for song:Before the Worst by The Script
-->Retrieving album name for Before the Worst by The Script
Info retrieved for song:Biological by Air
-->Retrieving album name for Biological by Air
Info retrieved for song:Foolish Games by Jewel
-->Retrieving album name for Foolish Games by Jewel
Info retrieved for song:(There Is) No Greater Love by Amy Winehouse
-->Retrieving album name for (There Is) No Greater Love by Amy Winehouse
Info retrieved for song:Sail Away by David Gray
-->Retrieving album name for Sail Away by David Gray
Info retrieved for song:My Love Is Your Love by Whitney Houston
-->Retrieving album name for My Love Is Your Love by Whitney Houston
Info retrieved for song:Mean by Taylor Swift
-->Retrieving album name for Mean by Taylor Swift
Info retrieved for song:Lay Me Down by Sam Smith
-->Retrieving album name for Lay Me Down by Sam Smith
Info retrieved for song:Fallin' for You by Colbie Caillat
-->Retrie

Info retrieved for song:If You Don't, Don't by Jimmy Eat World
-->Retrieving album name for If You Don't, Don't by Jimmy Eat World
Info retrieved for song:Song for Zula by Phosphorescent
-->Retrieving album name for Song for Zula by Phosphorescent
Info retrieved for song:Radio Cure by Wilco
-->Retrieving album name for Radio Cure by Wilco
Info retrieved for song:I Don't Believe You by P!nk
-->Retrieving album name for I Don't Believe You by P!nk
Info retrieved for song:A Letter To Elise by The Cure
-->Retrieving album name for A Letter To Elise by The Cure
Info retrieved for song:Atlantic by Keane
-->Retrieving album name for Atlantic by Keane
Info retrieved for song:Come Home by OneRepublic
-->Retrieving album name for Come Home by OneRepublic
Info retrieved for song:Keep Breathing by Ingrid Michaelson
-->Retrieving album name for Keep Breathing by Ingrid Michaelson
Info retrieved for song:My Cherie Amour by Stevie Wonder
-->Retrieving album name for My Cherie Amour by Stevie Wonder
I

-->Retrieving album name for Show Me the Meaning of Being Lonely by Backstreet Boys
Info retrieved for song:Big Girls Cry by Sia
-->Retrieving album name for Big Girls Cry by Sia
Info retrieved for song:Falling by Florence + the Machine
-->Retrieving album name for Falling by Florence + the Machine
Info retrieved for song:Back at One by Brian McKnight
-->Retrieving album name for Back at One by Brian McKnight
Info retrieved for song:Boat Behind by Kings of Convenience
-->Retrieving album name for Boat Behind by Kings of Convenience
Info retrieved for song:Stay by Lisa Loeb
-->Retrieving album name for Stay by Lisa Loeb
Info retrieved for song:A Taste of Honey by The Beatles
-->Retrieving album name for A Taste of Honey by The Beatles
Info retrieved for song:Atlas Hands by Benjamin Francis Leftwich
-->Retrieving album name for Atlas Hands by Benjamin Francis Leftwich
Info retrieved for song:Lights Out, Words Gone by Bombay Bicycle Club
-->Retrieving album name for Lights Out, Words Gone

-->Retrieving album name for Ain't Nobody (Loves Me Better) by Felix Jaehn
Info retrieved for song:By Your Side by CocoRosie
-->Retrieving album name for By Your Side by CocoRosie
Info retrieved for song:Coffee by Sylvan Esso
-->Retrieving album name for Coffee by Sylvan Esso
Info retrieved for song:Sorry Seems to Be the Hardest Word by Elton John
-->Retrieving album name for Sorry Seems to Be the Hardest Word by Elton John
Info retrieved for song:Wildest Moments by Jessie Ware
-->Retrieving album name for Wildest Moments by Jessie Ware
Info retrieved for song:Vermilion, Pt. 2 by Slipknot
-->Retrieving album name for Vermilion, Pt. 2 by Slipknot
----Missing field/result for Devotion by Slipknot. Skipping----
----Missing field/result for Vermilion, Pt. 2 by Slipknot. Skipping----
Info retrieved for song:Good Directions by Billy Currington
-->Retrieving album name for Good Directions by Billy Currington
Info retrieved for song:Rewind by Paolo Nutini
-->Retrieving album name for Rewind by

In [8]:
#Display dataframe and save as csv
top_songs_df.to_csv("csv/top_love_songs.csv")
top_songs_df

,Rank,Song Name,Artist,Duration,Album
0,1,Wonderwall,Oasis,259,(What's the Story) Morning Glory?
1,2,The Scientist,Coldplay,309,A Rush of Blood to the Head
2,3,Yellow,Coldplay,267,Parachutes
3,4,Fix You,Coldplay,294,X&Y
4,5,Chasing Cars,Snow Patrol,0,Eyes Open
...,...,...,...,...,...
995,996,Great Romances of the 20th Century,Taking Back Sunday,215,Tell All Your Friends
996,997,Surprise Ice,Kings of Convenience,293,Riot on an Empty Street
997,998,The Story,Brandi Carlile,306,The Story
998,999,One,Ed Sheeran,253,x


In [9]:
#Check if all rows of the df have been populated
#NOTE: not all songs have duration info (0). If we want to use duration to answer questions, we need to fill in the gaps
#NOTE: some albums are missing?
top_songs_df.count()  

Rank         1000
Song Name    1000
Artist       1000
Duration     1000
Album         970
dtype: int64

In [10]:
#MAYBE  GOOD TO DO JUST FOR THE PRESENTATION, DURATION IN SECONDS IS EASIER TO MANIPULATE FOR THE ANALYSIS
#Convert seconds in HH:MM:SS format
def convert(seconds): 
    seconds = seconds % (24 * 3600) 
#     hour = seconds // 3600    #we don't need hours
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
      
#     return "%d:%02d:%02d" % (hour, minutes, seconds)
    return "%02d:%02d" % (minutes, seconds)

# Example 
n = 259
print(convert(n))

04:19


In [11]:
###TRYING TO RETRIEVE RELEASE ALBUM INFO

##Test with Cher and Believe album as shown in LastFM website
# album_url=f'http://ws.audioscrobbler.com/2.0/?method=album.getinfo&api_key={lastfm_api}\
#             &artist=Cher&album=Believe&format=json'

# album_response=requests.get(album_url).json()
# pprint(album_response)    
